In [1]:
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 19.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}
print(PROJECT_ID)

qwiklabs-gcp-01-9506b6c36386


In [2]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

Initialized


In [3]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [4]:
def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.bbc_news_with_embeddings_table`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.embedding_model`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [5]:
def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [6]:
from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

In [7]:
QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

'Retrieved Data:'

'The US economy has grown more than expected, expanding at an annual rate of 3.8% in the last quarter of 2004.\n\nThe gross domestic product figure was ahead of the 3.1% the government estimated a month ago. The rise reflects stronger spending by businesses on capital equipment and a smaller-than-expected trade deficit. GDP is a measure of a country\'s economic health, reflecting the value of the goods and services it produces.\n\nThe new GDP figure, announced by the Commerce Department on Friday, also topped the 3.5% growth rate that economists had forecast ahead of Friday\'s announcement. Growth was at an annual rate of 4% in the third quarter of 2004 and for the year it came in at 4.4%, the best figure in five years. However, the positive economic climate may lead to a rise in interest rates, with many expecting US rates to rise on 22 March. In the January-to-March quarter, the economy is expected to grow at an annual rate of about 4%, economists forecast. In the final quarter of 20

' . . . '

'User Question:'

'Tell me about the US Economy'

'--------------------------------'

'Gemini Answer:'

'## The US Economy in 2004: A Strong Year with Concerns for the Future\n\nThe US economy grew at a robust pace in 2004, exceeding expectations with a 3.8% annual rate in the last quarter. This growth was driven by several factors, including:\n\n* **Stronger spending by businesses on capital equipment:** Businesses invested heavily in new equipment and software, contributing to the overall economic expansion.\n* **Smaller-than-expected trade deficit:** The trade deficit, which represents the difference between imports and exports, was lower than anticipated, boosting the economy.\n* **Increased consumer spending:** While consumer spending growth slowed slightly in the final quarter of 2004, it remained a significant contributor to economic growth throughout the year.\n\nHowever, despite the positive economic performance, several concerns remained:\n\n* **Rising interest rates:** The strong economy led to expectations of further interest rate increases by the Federal Reserve, which could